In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
'Good to check indivudual by get_symbol_info if status is trading'

'Good to check indivudual by get_symbol_info if status is trading'

In [6]:
'Filter the Info to get the list of Symbols ending with USDT'

'Filter the Info to get the list of Symbols ending with USDT'

In [7]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith('USDT')]

In [8]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [9]:
target_rank_info = [d['symbol']+'USDT' for d in rank_info if d['rank'] >= 300 and d['rank'] <= 500]

In [10]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [11]:
def is_candidate(filtered_klines_info, tolerable_volume):
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    trigger_kline = filtered_klines_info[-1]
    
    for context_kline in context_klines_info:
        
        volume_gate = float(context_kline[3])  <  tolerable_volume * 1.2
        if volume_gate :
            is_candidate = True
        else:
            return False


    volume_gate = float(trigger_kline[3]  > tolerable_volume * 10)

    if volume_gate:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [12]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [13]:
hit_ticker_dictionary = {}

In [14]:
full_test_data_set = []

for target_ticker in target_ticker_list:

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "7 day ago UTC") 
    length = int(len(klines))
    if(length < 24):
        break
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
    average_volume = np.mean(focused_kline_array[:,3])
    tolerable_volume = average_volume * 1.2
    
   
    for x in range(0, length):
        
 
        filtered_klines_info = focused_kline_array[x:x+24]
        buy_kline_info = filtered_klines_info[-1]
        if(len(filtered_klines_info) == 24 and is_candidate(filtered_klines_info, tolerable_volume)):
            
            
            print('********** SUCCESS ************************')
            print(target_ticker)
            print(x)
            print(buy_kline_info)
            print(klines[x:x+24][-1])
            
            key_test_data_set = [] 
            key_test_data_set.append(buy_kline_info[0])
            key_test_data_set.append
            
            
            hit_ticker_dictionary[target_ticker] = filtered_klines_info[-1:]
            

********** SUCCESS ************************
CTKUSDT
20
[1.65708360e+12 7.83000000e-01 8.52000000e-01 3.87478140e+06
 8.81226054e+00]
[1657083600000, '0.78300000', '0.86000000', '0.78300000', '0.85200000', '3874781.40000000', 1657087199999, '3210692.46980000', 13783, '1800085.70000000', '1488103.64990000', '0']
********** SUCCESS ************************
PHAUSDT
47
[1.65718080e+12 1.08600000e-01 1.14500000e-01 1.49206090e+07
 5.43278085e+00]
[1657180800000, '0.10860000', '0.12200000', '0.10860000', '0.11450000', '14920609.00000000', 1657184399999, '1748097.20570000', 5946, '9751277.00000000', '1145028.71460000', '0']


In [15]:
full_test_data_set = []

for key in hit_ticker_dictionary.keys():
    key_test_data_set = []
    key_test_data_set.append(key) 
    trigger_time_in_ms = hit_ticker_dictionary[key][0][0]
    key_test_data_set.append(trigger_time_in_ms) 
    open_price = hit_ticker_dictionary[key][0][4]
    key_test_data_set.append(open_price)
    analysis_kline_list = client.get_historical_klines(symbol=key, 
                                                       interval=Client.KLINE_INTERVAL_1HOUR, 
                                                       start_str=trigger_time_in_ms)[1:6]
    for sell_kline in analysis_kline_list:
        close_price = sell_kline[4]

        key_test_data_set.append(close_price)
        diff = (float(close_price) - float(open_price))
        profit = diff/float(open_price) * 100
        key_test_data_set.append(profit)
        
    full_test_data_set.append(key_test_data_set)
    

TypeError: Input type must be str

In [ ]:
full_test_data_set